## Imports

In [93]:
import numpy as np
import re
import pandas as pd
from config import config
from get_offers_by_city import connect

## Create Connection to de_rent Database

In [3]:
conn = connect(config())

Connecting to database...
Connection successful


## Load data from database

In [5]:
query = '''
    SELECT * FROM all_offers_infos
'''

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchall()

In [11]:
df = pd.DataFrame(result, columns=['offer_id', 'extraction_date', 'city', 'city_code', 'offer_type', 'offer_infos'])

In [13]:
df.head(1)

,offer_id,extraction_date,city,city_code,offer_type,offer_infos
0,45678593,2021-10-25 18:18:38,Dresden,100051,wohnung,"{""-----* frisch renovierte 3 Zimmerwohnung * I..."


## Data Cleaning

### Search for a pattern - separate the infos blocks. 

#### Cleaning one offer

In [139]:
all_info = df['offer_infos'][0].split(',')

for info in all_info:
    if 'Objektbeschreibung' in info:
        Objbeschreibung = info.split('-')
        Objbeschreibung = list(filter(lambda x: x != "", Objbeschreibung))
        print(Objbeschreibung)
    else:
        Objbeschreibung = np.nan


['"ObjektbeschreibungDiese 3 Zimmer Dachgeschosswohnung befindet sich in der Hardenbergstraße in direkter Innenstadtlage.In den Wohnräumen liegt hochwertiger Laminatboden. Die Wohnung wird renoviert übergeben. Die Wohnung befindet sich 4. Stock. Es ist kein Aufzug vorhanden !!"']


In [162]:
for info in all_info:
    if 'Anbieter' in info:
        print(info)

"Immonet-Nr.: 45678593Anbieter-Objekt-ID: A8BJ23K_2SS5K4U-1979606"
"AnbieterIhr Ansprechpartner---------Violetta Ilves-----------------Schöne Aussicht 42--------------------------65193 Wiesbaden--------Ihr Anbieter H & I Invest GmbH---------- Frau Ilves-----------------Schöne Aussicht 42-----------------65193 Wiesbaden--------Anbieter kontaktierenAnsprechpartnerTelefonnummer------------Tel.:------------0176 - 313 93007-----------------------Mobil:------------0176 - 313 93007-----------AnbieterTelefonnummerMobil: 0176-31393007Weitere Objekte des AnbietersHomepage des AnbietersAnbieterimpressum"}


In [166]:
df_list = []

for offer in range(len(df)):
    # All info pannels separated in a list
    all_info = df['offer_infos'][offer].split(',')
    
    offer_infos_list = []
    offer_infos_list.append({'offer_id': df['offer_id'][offer]})
    offer_infos_list.append({'city': df['city'][offer]})
    offer_infos_list.append({'offer_type': df['offer_type'][offer]})
    # Preise & Kosten
    for info in all_info:
        if 'Preise & Kosten' in info:
            info = info.split('.0')
            for i in info:
                if 'Miete' in i:
                    miete = re.findall(r'\d+', i)
                    #print(miete)
                else:
                    miete = np.nan
                offer_infos_list.append({'miete':miete})
                if 'Nebenkosten' in i:
                    nbkosten = re.findall(r'\d+', i)
                    #print(nbkosten)
                else:
                    nbkosten = np.nan
                offer_infos_list.append({'nbkosten':nbkosten})
                if 'Heizkosten' in i:
                    hzkosten = re.findall(r'\d+', i)
                    #print(hzkosten)
                else:
                    hzkosten = np.nan
                offer_infos_list.append({'hzkosten':hzkosten})
    # kaution
    #for info in all_info:
        if 'Kaution' in info:
            kaution = re.findall('\w+:.*', info) 
        else:
            kaution = np.nan
        offer_infos_list.append({'kaution':kaution})

    # Größe & Zustand
    #for info in all_info:
        if 'Größe & Zustand' in info:
            info = re.split('(?=[A-Z])', info)
            for i in info:
                if 'Zimmer' in i:
                    zimmer = re.findall(r'\d+', i)[0]
                    #print(zimmer)
                else: 
                    zimmer = np.nan
                offer_infos_list.append({'zimmer':zimmer})
                if 'Wohnfläche' in i:
                    wflache = re.findall(r'\d+', i)[0]
                    #print(wflache)
                else:
                    wflache = np.nan
                offer_infos_list.append({'wflache':wflache})
                if 'Baujahr' in i:
                    baujahr = re.findall(r'\d+', i)[0]
                    #print(baujahr)
                else: 
                    baujahr = np.nan
                offer_infos_list.append({'baujahr':baujahr})
                if 'Verfügbar' in i:
                    verfugbar = i.split(' ')[1].replace('-', '')
                    #print(verfugbar)
                else: 
                    verfugbar = np.nan
                offer_infos_list.append({'verfugbar':verfugbar})

    # Energie

    # Ausstattung
    #for info in all_info:
        if 'Ausstattung' in info:
            info = info.split('-')
            ausstattung = list(filter(lambda x: x != "", info))
            ausstattung = [i.replace('"', '') for i in ausstattung]
            #print(ausstattung)
        else:
            ausstattung = np.nan
        offer_infos_list.append({'ausstattung':ausstattung})

    # Objektbeschreibung
    #for info in all_info:
        if 'Objektbeschreibung' in info:
            Objbeschreibung = info.split('-')
            Objbeschreibung = list(filter(lambda x: x != "", Objbeschreibung))
            #print(Objbeschreibung)
        else:
            Objbeschreibung = np.nan
        offer_infos_list.append({'Objbeschreibung':Objbeschreibung})    
    # Lage

    # Sonstiges
    #for info in all_info:
        if 'Sonstiges' in info:
            sortiges = info
        else:
            sortiges = np.nan
        offer_infos_list.append({'sortiges':sortiges})    

    # Anbieter
    
    # append infos from the offer
    df_list.append(offer_infos_list)

df_cleaned = pd.DataFrame(df_list)
df_cleaned

,0,1,2,3,4,5,6,7,8,9,...,572,573,574,575,576,577,578,579,580,581
0,{'offer_id': 45678593},{'city': 'Dresden'},{'offer_type': 'wohnung'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},...,None,None,None,None,None,None,None,None,None,None
1,{'offer_id': 45514753},{'city': 'Dresden'},{'offer_type': 'wohnung'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,{'offer_id': 44924940},{'city': 'Dresden'},{'offer_type': 'wohnung'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'miete': ['440']},{'nbkosten': nan},{'hzkosten': nan},...,None,None,None,None,None,None,None,None,None,None
3,{'offer_id': 44630030},{'city': 'Dresden'},{'offer_type': 'wohnung'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},...,None,None,None,None,None,None,None,None,None,None
4,{'offer_id': 45514780},{'city': 'Dresden'},{'offer_type': 'wohnung'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6756,{'offer_id': 45645806},{'city': 'Hamburg'},{'offer_type': 'haus'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'miete': ['1424']},{'nbkosten': nan},{'hzkosten': nan},...,None,None,None,None,None,None,None,None,None,None
6757,{'offer_id': 45645808},{'city': 'Hamburg'},{'offer_type': 'haus'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'miete': ['1584']},{'nbkosten': nan},{'hzkosten': nan},...,None,None,None,None,None,None,None,None,None,None
6758,{'offer_id': 45318130},{'city': 'Hamburg'},{'offer_type': 'haus'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'miete': ['452']},{'nbkosten': nan},{'hzkosten': nan},...,None,None,None,None,None,None,None,None,None,None
6759,{'offer_id': 45318133},{'city': 'Hamburg'},{'offer_type': 'haus'},{'kaution': nan},{'ausstattung': nan},{'Objbeschreibung': nan},{'sortiges': nan},{'miete': ['540']},{'nbkosten': nan},{'hzkosten': nan},...,None,None,None,None,None,None,None,None,None,None


In [167]:
df_list[0]

[{'offer_id': 45678593},
 {'city': 'Dresden'},
 {'offer_type': 'wohnung'},
 {'kaution': nan},
 {'ausstattung': nan},
 {'Objbeschreibung': nan},
 {'sortiges': nan},
 {'kaution': nan},
 {'ausstattung': nan},
 {'Objbeschreibung': nan},
 {'sortiges': nan},
 {'miete': ['790']},
 {'nbkosten': nan},
 {'hzkosten': nan},
 {'miete': nan},
 {'nbkosten': ['80']},
 {'hzkosten': nan},
 {'miete': nan},
 {'nbkosten': nan},
 {'hzkosten': ['80']},
 {'miete': nan},
 {'nbkosten': nan},
 {'hzkosten': nan},
 {'kaution': nan},
 {'ausstattung': nan},
 {'Objbeschreibung': nan},
 {'sortiges': nan},
 {'kaution': nan},
 {'zimmer': nan},
 {'wflache': nan},
 {'baujahr': nan},
 {'verfugbar': nan},
 {'zimmer': nan},
 {'wflache': nan},
 {'baujahr': nan},
 {'verfugbar': nan},
 {'zimmer': nan},
 {'wflache': nan},
 {'baujahr': nan},
 {'verfugbar': nan},
 {'zimmer': '3'},
 {'wflache': nan},
 {'baujahr': nan},
 {'verfugbar': nan},
 {'zimmer': nan},
 {'wflache': '65'},
 {'baujahr': nan},
 {'verfugbar': nan},
 {'zimmer': nan